# Parameter Estimation Workshop
## Method of Moments & Maximum Likelihood Estimation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/CMSC173/blob/main/08-ParameterEstimation/notebooks/parameter_estimation_workshop.ipynb)

**Course:** CMSC 173 - Machine Learning  
**Topic:** Parameter Estimation  
**Duration:** 45-60 minutes  

### Learning Objectives
By the end of this workshop, you will be able to:
1. Understand the fundamental concepts of parameter estimation
2. Implement Method of Moments (MoM) estimators
3. Implement Maximum Likelihood Estimation (MLE)
4. Compare different estimation methods
5. Apply parameter estimation to real-world problems
6. Evaluate estimator quality and diagnostics

## Setup & Imports

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, optimize
from sklearn.datasets import make_regression
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Setup complete! All libraries imported successfully.")

## Part 1: Problem Understanding

Parameter estimation is the process of inferring unknown parameters of a probability distribution from observed data.

**The Setup:**
- We have data samples: $\{x_1, x_2, \ldots, x_n\}$
- We assume they come from a distribution: $f(x|\theta)$
- We want to estimate: $\hat{\theta}$

In [ ]:
# Generate example data from a known distribution
true_mu = 5.0
true_sigma = 2.0
n_samples = 100

# Generate normal data
data = np.random.normal(true_mu, true_sigma, n_samples)

print(f"Generated {n_samples} samples from Normal({true_mu}, {true_sigma}²)")
print(f"Sample mean: {np.mean(data):.3f}")
print(f"Sample std: {np.std(data, ddof=1):.3f}")

# Visualize the data
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
plt.hist(data, bins=20, density=True, alpha=0.7, color='lightblue', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Sample Data')
plt.grid(True, alpha=0.3)

# Box plot
plt.subplot(1, 2, 2)
plt.boxplot(data, vert=True)
plt.ylabel('Value')
plt.title('Data Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Notice: We generated this data, so we know the true parameters.")
print("   In practice, we only have the data and need to estimate the parameters!")

## Part 2: Method of Moments (MoM)

**Core Idea:** Match sample moments to theoretical moments.

**Algorithm:**
1. Express theoretical moments in terms of parameters
2. Calculate sample moments
3. Set theoretical = sample moments
4. Solve for parameters

In [ ]:
def method_of_moments_normal(data):
    """
    Method of Moments estimation for normal distribution
    
    For Normal(μ, σ²):
    - E[X] = μ
    - E[X²] = μ² + σ²
    
    Sample moments:
    - m₁ = (1/n) Σ xᵢ
    - m₂ = (1/n) Σ xᵢ²
    """
    # First moment (mean)
    m1 = np.mean(data)
    
    # Second moment
    m2 = np.mean(data**2)
    
    # MoM estimates
    mu_hat = m1
    sigma_squared_hat = m2 - m1**2
    sigma_hat = np.sqrt(sigma_squared_hat)
    
    return mu_hat, sigma_hat

# Apply MoM to our data
mu_mom, sigma_mom = method_of_moments_normal(data)

print("Method of Moments Results:")
print(f"μ̂ = {mu_mom:.3f} (true μ = {true_mu})")
print(f"σ̂ = {sigma_mom:.3f} (true σ = {true_sigma})")
print(f"Error in μ: {abs(mu_mom - true_mu):.3f}")
print(f"Error in σ: {abs(sigma_mom - true_sigma):.3f}")

In [ ]:
# Visualize MoM fit
x_range = np.linspace(data.min() - 1, data.max() + 1, 200)

plt.figure(figsize=(10, 6))
plt.hist(data, bins=20, density=True, alpha=0.7, color='lightblue', 
         edgecolor='black', label='Data')

# True distribution
true_pdf = stats.norm.pdf(x_range, true_mu, true_sigma)
plt.plot(x_range, true_pdf, 'r-', linewidth=2, 
         label=f'True: N({true_mu}, {true_sigma}²)')

# MoM fitted distribution
mom_pdf = stats.norm.pdf(x_range, mu_mom, sigma_mom)
plt.plot(x_range, mom_pdf, 'g--', linewidth=2, 
         label=f'MoM: N({mu_mom:.2f}, {sigma_mom:.2f}²)')

plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Method of Moments: Normal Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("💡 The MoM estimate closely matches the true distribution!")

### MoM Example: Poisson Distribution

For Poisson(λ): E[X] = λ, so the MoM estimator is simply λ̂ = x̄

In [ ]:
# Generate Poisson data
true_lambda = 3.5
poisson_data = np.random.poisson(true_lambda, 200)

# MoM estimate
lambda_mom = np.mean(poisson_data)

print(f"Poisson MoM Estimation:")
print(f"True λ = {true_lambda}")
print(f"MoM λ̂ = {lambda_mom:.3f}")
print(f"Error: {abs(lambda_mom - true_lambda):.3f}")

# Visualize
plt.figure(figsize=(10, 6))

# Data histogram
counts = np.bincount(poisson_data)
x_vals = np.arange(len(counts))
plt.bar(x_vals, counts/len(poisson_data), alpha=0.7, color='lightcoral', 
        edgecolor='black', label='Data')

# Theoretical PMFs
x_theory = np.arange(0, max(poisson_data) + 1)
true_pmf = stats.poisson.pmf(x_theory, true_lambda)
mom_pmf = stats.poisson.pmf(x_theory, lambda_mom)

plt.plot(x_theory, true_pmf, 'ro-', markersize=4, 
         label=f'True: Poisson({true_lambda})')
plt.plot(x_theory, mom_pmf, 'g^-', markersize=4, 
         label=f'MoM: Poisson({lambda_mom:.2f})')

plt.xlabel('Value')
plt.ylabel('Probability')
plt.title('Method of Moments: Poisson Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 3: Maximum Likelihood Estimation (MLE)

**Core Idea:** Find parameter values that make the observed data most likely.

**Likelihood Function:**
$$L(\theta) = \prod_{i=1}^n f(x_i | \theta)$$

**Log-Likelihood:**
$$\ell(\theta) = \sum_{i=1}^n \log f(x_i | \theta)$$

**MLE:** $\hat{\theta}_{MLE} = \arg\max_\theta \ell(\theta)$

In [ ]:
def maximum_likelihood_normal(data):
    """
    Maximum Likelihood estimation for normal distribution
    
    For Normal(μ, σ²), the MLE estimates are:
    - μ̂ = (1/n) Σ xᵢ
    - σ̂² = (1/n) Σ (xᵢ - μ̂)²
    """
    n = len(data)
    
    # MLE for mean
    mu_mle = np.mean(data)
    
    # MLE for variance (biased estimator)
    sigma_squared_mle = np.sum((data - mu_mle)**2) / n
    sigma_mle = np.sqrt(sigma_squared_mle)
    
    return mu_mle, sigma_mle

# Apply MLE to our data
mu_mle, sigma_mle = maximum_likelihood_normal(data)

print("Maximum Likelihood Results:")
print(f"μ̂ = {mu_mle:.3f} (true μ = {true_mu})")
print(f"σ̂ = {sigma_mle:.3f} (true σ = {true_sigma})")
print(f"Error in μ: {abs(mu_mle - true_mu):.3f}")
print(f"Error in σ: {abs(sigma_mle - true_sigma):.3f}")

print("\n📊 Comparison:")
print(f"MoM μ̂: {mu_mom:.3f}, MLE μ̂: {mu_mle:.3f}")
print(f"MoM σ̂: {sigma_mom:.3f}, MLE σ̂: {sigma_mle:.3f}")
print("\n💡 For normal distribution, MoM and MLE give the same estimate for μ!")
print("   But σ estimates differ slightly (MLE uses n, MoM effectively uses n-1)")

In [ ]:
# Visualize the likelihood function
def log_likelihood_normal(mu, sigma, data):
    """Compute log-likelihood for normal distribution"""
    n = len(data)
    if sigma <= 0:
        return -np.inf
    
    ll = -0.5 * n * np.log(2 * np.pi) - n * np.log(sigma) - \
         0.5 * np.sum((data - mu)**2) / (sigma**2)
    return ll

# Create grid for likelihood surface
mu_range = np.linspace(3, 7, 50)
sigma_range = np.linspace(1, 3, 50)
MU, SIGMA = np.meshgrid(mu_range, sigma_range)

# Compute log-likelihood surface
log_lik_surface = np.zeros_like(MU)
for i in range(len(mu_range)):
    for j in range(len(sigma_range)):
        log_lik_surface[j, i] = log_likelihood_normal(MU[j, i], SIGMA[j, i], data)

# Plot
plt.figure(figsize=(12, 5))

# 2D contour plot
plt.subplot(1, 2, 1)
contour = plt.contour(MU, SIGMA, log_lik_surface, levels=20, colors='blue', alpha=0.6)
plt.clabel(contour, inline=True, fontsize=8)
plt.plot(true_mu, true_sigma, 'ro', markersize=8, label='True Parameters')
plt.plot(mu_mle, sigma_mle, 'go', markersize=8, label='MLE Estimates')
plt.xlabel('μ')
plt.ylabel('σ')
plt.title('2D Log-Likelihood Surface')
plt.legend()
plt.grid(True, alpha=0.3)

# 1D slice (fix sigma at true value)
plt.subplot(1, 2, 2)
log_lik_1d = [log_likelihood_normal(mu, true_sigma, data) for mu in mu_range]
plt.plot(mu_range, log_lik_1d, 'b-', linewidth=2, label='Log-Likelihood')
plt.axvline(true_mu, color='red', linestyle='--', linewidth=2, label='True μ')
plt.axvline(mu_mle, color='green', linestyle=':', linewidth=2, label='MLE μ̂')
plt.xlabel('μ')
plt.ylabel('Log-Likelihood')
plt.title('1D Log-Likelihood (σ fixed)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### MLE Example: Exponential Distribution

For Exponential(λ): f(x|λ) = λe^{-λx}, the MLE is λ̂ = 1/x̄

In [ ]:
# Generate exponential data
true_rate = 2.0
exp_data = np.random.exponential(1/true_rate, 100)

# MLE estimate
rate_mle = 1 / np.mean(exp_data)

print(f"Exponential MLE Estimation:")
print(f"True λ = {true_rate}")
print(f"MLE λ̂ = {rate_mle:.3f}")
print(f"Error: {abs(rate_mle - true_rate):.3f}")

# Visualize
plt.figure(figsize=(12, 5))

# Data and fitted distributions
plt.subplot(1, 2, 1)
plt.hist(exp_data, bins=20, density=True, alpha=0.7, color='lightcoral', 
         edgecolor='black', label='Data')

x_exp = np.linspace(0, max(exp_data), 200)
true_pdf = stats.expon.pdf(x_exp, scale=1/true_rate)
mle_pdf = stats.expon.pdf(x_exp, scale=1/rate_mle)

plt.plot(x_exp, true_pdf, 'r-', linewidth=2, label=f'True: Exp({true_rate})')
plt.plot(x_exp, mle_pdf, 'g--', linewidth=2, label=f'MLE: Exp({rate_mle:.2f})')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Exponential Distribution MLE')
plt.legend()
plt.grid(True, alpha=0.3)

# Log-likelihood function
plt.subplot(1, 2, 2)
rate_range = np.linspace(0.5, 4, 100)
log_lik_exp = []

for rate in rate_range:
    ll = np.sum(stats.expon.logpdf(exp_data, scale=1/rate))
    log_lik_exp.append(ll)

plt.plot(rate_range, log_lik_exp, 'b-', linewidth=2, label='Log-Likelihood')
plt.axvline(true_rate, color='red', linestyle='--', linewidth=2, label='True λ')
plt.axvline(rate_mle, color='green', linestyle=':', linewidth=2, label='MLE λ̂')
plt.xlabel('λ (Rate Parameter)')
plt.ylabel('Log-Likelihood')
plt.title('Log-Likelihood Function')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Advanced Techniques

### Numerical MLE for Complex Distributions

When there's no closed-form solution, we use numerical optimization.

In [ ]:
# Example: Gamma distribution (requires numerical optimization)
from scipy.optimize import minimize

# Generate gamma data
true_shape = 2.5
true_scale = 1.5
gamma_data = np.random.gamma(true_shape, true_scale, 200)

def negative_log_likelihood_gamma(params, data):
    """Negative log-likelihood for gamma distribution"""
    shape, scale = params
    if shape <= 0 or scale <= 0:
        return np.inf
    
    try:
        ll = np.sum(stats.gamma.logpdf(data, a=shape, scale=scale))
        return -ll  # Return negative for minimization
    except:
        return np.inf

# Initial guess
initial_guess = [2.0, 1.0]

# Optimize
result = minimize(negative_log_likelihood_gamma, initial_guess, 
                 args=(gamma_data,), method='Nelder-Mead')

shape_mle, scale_mle = result.x

print("Gamma Distribution Numerical MLE:")
print(f"True: shape={true_shape}, scale={true_scale}")
print(f"MLE:  shape={shape_mle:.3f}, scale={scale_mle:.3f}")
print(f"Optimization success: {result.success}")
print(f"Function evaluations: {result.nfev}")

# Compare with Method of Moments
sample_mean = np.mean(gamma_data)
sample_var = np.var(gamma_data)

# For Gamma: E[X] = αβ, Var(X) = αβ²
scale_mom = sample_var / sample_mean
shape_mom = sample_mean / scale_mom

print("\nMethod of Moments:")
print(f"MoM:  shape={shape_mom:.3f}, scale={scale_mom:.3f}")

# Visualize results
plt.figure(figsize=(10, 6))
plt.hist(gamma_data, bins=25, density=True, alpha=0.7, color='lightblue', 
         edgecolor='black', label='Data')

x_gamma = np.linspace(0, max(gamma_data), 200)

# True distribution
true_pdf = stats.gamma.pdf(x_gamma, a=true_shape, scale=true_scale)
plt.plot(x_gamma, true_pdf, 'r-', linewidth=2, 
         label=f'True: Γ({true_shape}, {true_scale})')

# MLE fitted
mle_pdf = stats.gamma.pdf(x_gamma, a=shape_mle, scale=scale_mle)
plt.plot(x_gamma, mle_pdf, 'g--', linewidth=2, 
         label=f'MLE: Γ({shape_mle:.2f}, {scale_mle:.2f})')

# MoM fitted
mom_pdf = stats.gamma.pdf(x_gamma, a=shape_mom, scale=scale_mom)
plt.plot(x_gamma, mom_pdf, 'b:', linewidth=2, 
         label=f'MoM: Γ({shape_mom:.2f}, {scale_mom:.2f})')

plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Numerical MLE: Gamma Distribution')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Part 5: Diagnostic Tools

How do we know if our parameter estimates are good?

In [ ]:
# Bootstrap confidence intervals
def bootstrap_confidence_interval(data, estimator_func, n_bootstrap=1000, confidence=0.95):
    """Compute bootstrap confidence interval for an estimator"""
    estimates = []
    
    for _ in range(n_bootstrap):
        # Bootstrap sample
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
        estimate = estimator_func(bootstrap_sample)
        estimates.append(estimate)
    
    estimates = np.array(estimates)
    alpha = 1 - confidence
    lower = np.percentile(estimates, 100 * alpha/2)
    upper = np.percentile(estimates, 100 * (1 - alpha/2))
    
    return estimates, (lower, upper)

# Apply to our normal data
def mean_estimator(data):
    return np.mean(data)

mean_bootstrap, mean_ci = bootstrap_confidence_interval(data, mean_estimator)

print(f"Bootstrap Results for Mean Estimation:")
print(f"Original estimate: {np.mean(data):.3f}")
print(f"Bootstrap mean: {np.mean(mean_bootstrap):.3f}")
print(f"Bootstrap std: {np.std(mean_bootstrap):.3f}")
print(f"95% CI: [{mean_ci[0]:.3f}, {mean_ci[1]:.3f}]")
print(f"True value in CI: {mean_ci[0] <= true_mu <= mean_ci[1]}")

# Visualize bootstrap distribution
plt.figure(figsize=(12, 5))

# Bootstrap distribution
plt.subplot(1, 2, 1)
plt.hist(mean_bootstrap, bins=30, density=True, alpha=0.7, color='lightgreen', 
         edgecolor='black', label='Bootstrap Distribution')
plt.axvline(true_mu, color='red', linestyle='--', linewidth=2, label='True μ')
plt.axvline(np.mean(data), color='blue', linestyle='-', linewidth=2, label='Original Estimate')
plt.axvline(mean_ci[0], color='orange', linestyle=':', linewidth=2, label='95% CI')
plt.axvline(mean_ci[1], color='orange', linestyle=':', linewidth=2)
plt.fill_between([mean_ci[0], mean_ci[1]], 0, plt.ylim()[1], alpha=0.2, color='orange')
plt.xlabel('Mean Estimate')
plt.ylabel('Density')
plt.title('Bootstrap Distribution of Mean')
plt.legend()
plt.grid(True, alpha=0.3)

# Q-Q plot for normality check
plt.subplot(1, 2, 2)
from scipy import stats
(osm, osr), (slope, intercept, r) = stats.probplot(data, dist="norm", plot=None)
plt.scatter(osm, osr, alpha=0.6, color='blue', s=20)
plt.plot(osm, slope * osm + intercept, 'r-', linewidth=2, 
         label=f'Normal Q-Q Line (R² = {r**2:.3f})')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
plt.title('Normality Check (Q-Q Plot)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Goodness-of-Fit Testing

In [ ]:
# Kolmogorov-Smirnov test
from scipy.stats import kstest

# Test if our data comes from the estimated normal distribution
ks_stat, ks_pvalue = kstest(data, lambda x: stats.norm.cdf(x, mu_mle, sigma_mle))

print("Goodness-of-Fit Tests:")
print(f"Kolmogorov-Smirnov Test:")
print(f"  Statistic: {ks_stat:.4f}")
print(f"  P-value: {ks_pvalue:.4f}")
print(f"  Conclusion: {'Reject H0' if ks_pvalue < 0.05 else 'Fail to reject H0'} (α=0.05)")

# Anderson-Darling test for normality
from scipy.stats import anderson
ad_stat, ad_critical, ad_significance = anderson(data, dist='norm')

print(f"\nAnderson-Darling Test for Normality:")
print(f"  Statistic: {ad_stat:.4f}")
for i, (crit, sig) in enumerate(zip(ad_critical, ad_significance)):
    print(f"  Critical value at {sig}%: {crit:.4f} {'(Reject)' if ad_stat > crit else '(Accept)'}")

# Visualize fit
plt.figure(figsize=(12, 5))

# Empirical vs theoretical CDF
plt.subplot(1, 2, 1)
sorted_data = np.sort(data)
empirical_cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
theoretical_cdf = stats.norm.cdf(sorted_data, mu_mle, sigma_mle)

plt.plot(sorted_data, empirical_cdf, 'b-', linewidth=2, label='Empirical CDF')
plt.plot(sorted_data, theoretical_cdf, 'r--', linewidth=2, label='Theoretical CDF')
plt.xlabel('Value')
plt.ylabel('Cumulative Probability')
plt.title('CDF Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# P-P plot
plt.subplot(1, 2, 2)
plt.scatter(theoretical_cdf, empirical_cdf, alpha=0.6, s=20)
plt.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Perfect Fit')
plt.xlabel('Theoretical Cumulative Probability')
plt.ylabel('Empirical Cumulative Probability')
plt.title('P-P Plot')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 If the points lie close to the diagonal line, the fit is good!")

## Part 6: Best Practices

### Comparing Estimator Performance

In [ ]:
# Simulation study to compare MoM vs MLE
def compare_estimators(true_params, distribution, sample_sizes, n_simulations=500):
    """Compare MoM and MLE estimators across different sample sizes"""
    results = {
        'sample_sizes': sample_sizes,
        'mom_bias': [],
        'mle_bias': [],
        'mom_variance': [],
        'mle_variance': [],
        'mom_mse': [],
        'mle_mse': []
    }
    
    for n in sample_sizes:
        mom_estimates = []
        mle_estimates = []
        
        for _ in range(n_simulations):
            # Generate data
            if distribution == 'normal':
                data_sim = np.random.normal(true_params[0], true_params[1], n)
                mom_est = np.std(data_sim, ddof=1)  # Unbiased variance estimator
                mle_est = np.std(data_sim, ddof=0)  # MLE variance estimator
            elif distribution == 'exponential':
                data_sim = np.random.exponential(1/true_params[0], n)
                mom_est = 1/np.mean(data_sim)  # Same as MLE for exponential
                mle_est = 1/np.mean(data_sim)
            
            mom_estimates.append(mom_est)
            mle_estimates.append(mle_est)
        
        # Calculate statistics
        true_param = true_params[1] if distribution == 'normal' else true_params[0]
        
        mom_estimates = np.array(mom_estimates)
        mle_estimates = np.array(mle_estimates)
        
        mom_bias = np.mean(mom_estimates) - true_param
        mle_bias = np.mean(mle_estimates) - true_param
        
        mom_var = np.var(mom_estimates)
        mle_var = np.var(mle_estimates)
        
        mom_mse = mom_bias**2 + mom_var
        mle_mse = mle_bias**2 + mle_var
        
        results['mom_bias'].append(mom_bias)
        results['mle_bias'].append(mle_bias)
        results['mom_variance'].append(mom_var)
        results['mle_variance'].append(mle_var)
        results['mom_mse'].append(mom_mse)
        results['mle_mse'].append(mle_mse)
    
    return results

# Run comparison for normal distribution (comparing variance estimators)
sample_sizes = [10, 20, 50, 100, 200, 500]
results = compare_estimators([5.0, 2.0], 'normal', sample_sizes)

# Plot results
plt.figure(figsize=(15, 5))

# Bias
plt.subplot(1, 3, 1)
plt.plot(sample_sizes, results['mom_bias'], 'bo-', label='MoM (Unbiased)')
plt.plot(sample_sizes, results['mle_bias'], 'ro-', label='MLE (Biased)')
plt.axhline(0, color='black', linestyle='--', alpha=0.5)
plt.xlabel('Sample Size')
plt.ylabel('Bias')
plt.title('Bias Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Variance
plt.subplot(1, 3, 2)
plt.loglog(sample_sizes, results['mom_variance'], 'bo-', label='MoM')
plt.loglog(sample_sizes, results['mle_variance'], 'ro-', label='MLE')
plt.xlabel('Sample Size')
plt.ylabel('Variance (log scale)')
plt.title('Variance Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# MSE
plt.subplot(1, 3, 3)
plt.loglog(sample_sizes, results['mom_mse'], 'bo-', label='MoM')
plt.loglog(sample_sizes, results['mle_mse'], 'ro-', label='MLE')
plt.xlabel('Sample Size')
plt.ylabel('MSE (log scale)')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Key Observations:")
print("• MoM (unbiased) estimator has zero bias")
print("• MLE estimator is biased but has lower variance")
print("• For large samples, MLE bias becomes negligible")
print("• MSE reflects the bias-variance tradeoff")

## Student Activity (15 minutes)

### 🎯 Your Turn: Parameter Estimation Challenge

You're given a dataset and need to:
1. Identify the most likely distribution
2. Estimate parameters using both MoM and MLE
3. Compare the methods
4. Validate your results

In [ ]:
# Mystery dataset - what distribution does this come from?
np.random.seed(123)  # Different seed for the activity
mystery_data = np.array([
    2.3, 1.8, 3.1, 2.7, 1.9, 4.2, 2.1, 3.8, 2.9, 1.6,
    3.5, 2.4, 1.7, 3.9, 2.8, 2.2, 4.1, 1.5, 3.3, 2.6,
    1.9, 3.7, 2.5, 2.0, 3.4, 2.7, 1.8, 4.0, 2.3, 3.2,
    2.1, 2.9, 1.7, 3.6, 2.4, 2.8, 3.1, 1.6, 3.8, 2.2,
    2.6, 1.9, 3.3, 2.7, 2.0, 3.5, 2.5, 1.8, 4.2, 2.9
])

print("🔍 Mystery Dataset Analysis")
print(f"Sample size: {len(mystery_data)}")
print(f"Sample mean: {np.mean(mystery_data):.3f}")
print(f"Sample std: {np.std(mystery_data, ddof=1):.3f}")
print(f"Sample min: {np.min(mystery_data):.3f}")
print(f"Sample max: {np.max(mystery_data):.3f}")

# Visualize the data
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.hist(mystery_data, bins=10, density=True, alpha=0.7, color='lightpink', edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Histogram')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.boxplot(mystery_data)
plt.ylabel('Value')
plt.title('Box Plot')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
stats.probplot(mystery_data, dist="norm", plot=plt)
plt.title('Q-Q Plot (Normal)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📝 Your Tasks:")
print("1. Based on the visualizations, what distribution do you think this is?")
print("2. Estimate parameters using Method of Moments")
print("3. Estimate parameters using Maximum Likelihood")
print("4. Compare your estimates")
print("5. Test goodness of fit")
print("\n⏰ You have 15 minutes. Work in the cells below!")

In [ ]:
# 🚀 YOUR CODE HERE - Task 1: Hypothesis about distribution
# Look at the histogram, box plot, and Q-Q plot
# What distribution family do you think this data comes from?

print("My hypothesis: The data appears to come from a _______ distribution")
print("Reasoning: ")
# Your reasoning here

In [ ]:
# 🚀 YOUR CODE HERE - Task 2: Method of Moments estimation
# Assuming normal distribution, estimate μ and σ using MoM

# Calculate sample moments
# Your code here

# MoM estimates
# Your code here

print(f"Method of Moments estimates:")
print(f"μ̂_MoM = ")
print(f"σ̂_MoM = ")

In [ ]:
# 🚀 YOUR CODE HERE - Task 3: Maximum Likelihood estimation
# Estimate μ and σ using MLE

# MLE estimates for normal distribution
# Your code here

print(f"Maximum Likelihood estimates:")
print(f"μ̂_MLE = ")
print(f"σ̂_MLE = ")

In [ ]:
# 🚀 YOUR CODE HERE - Task 4: Compare estimates and visualize fit
# Plot the data with both fitted distributions

# Your visualization code here

print("Comparison of estimates:")
print(f"Difference in μ estimates: ")
print(f"Difference in σ estimates: ")

In [ ]:
# 🚀 YOUR CODE HERE - Task 5: Goodness of fit test
# Use Kolmogorov-Smirnov test or similar

# Your testing code here

print("Goodness of fit results:")
print(f"Test statistic: ")
print(f"P-value: ")
print(f"Conclusion: ")

## Solutions (Hidden - Reveal After Activity)

<details>
<summary>Click to reveal solutions</summary>

### Solution Code:

In [ ]:
# SOLUTION - Task 1: Distribution identification
print("✅ SOLUTION - Task 1:")
print("The data appears to come from a NORMAL distribution")
print("Reasoning:")
print("• Histogram shows roughly bell-shaped distribution")
print("• Box plot shows symmetric distribution with no extreme outliers")
print("• Q-Q plot points lie approximately on the diagonal line")
print("• Data range is continuous and symmetric around the mean")

In [ ]:
# SOLUTION - Task 2: Method of Moments
print("\n✅ SOLUTION - Task 2:")

# For normal distribution: E[X] = μ, Var(X) = σ²
mu_mom_solution = np.mean(mystery_data)
sigma_mom_solution = np.std(mystery_data, ddof=1)  # Using unbiased estimator

print(f"Method of Moments estimates:")
print(f"μ̂_MoM = {mu_mom_solution:.3f}")
print(f"σ̂_MoM = {sigma_mom_solution:.3f}")

In [ ]:
# SOLUTION - Task 3: Maximum Likelihood
print("\n✅ SOLUTION - Task 3:")

# For normal distribution MLE
mu_mle_solution = np.mean(mystery_data)
sigma_mle_solution = np.std(mystery_data, ddof=0)  # MLE uses n in denominator

print(f"Maximum Likelihood estimates:")
print(f"μ̂_MLE = {mu_mle_solution:.3f}")
print(f"σ̂_MLE = {sigma_mle_solution:.3f}")

In [ ]:
# SOLUTION - Task 4: Comparison and visualization
print("\n✅ SOLUTION - Task 4:")

plt.figure(figsize=(10, 6))
plt.hist(mystery_data, bins=12, density=True, alpha=0.7, color='lightpink', 
         edgecolor='black', label='Data')

x_range = np.linspace(mystery_data.min() - 0.5, mystery_data.max() + 0.5, 200)

# MoM fitted distribution
mom_pdf = stats.norm.pdf(x_range, mu_mom_solution, sigma_mom_solution)
plt.plot(x_range, mom_pdf, 'b-', linewidth=2, 
         label=f'MoM: N({mu_mom_solution:.2f}, {sigma_mom_solution:.2f}²)')

# MLE fitted distribution
mle_pdf = stats.norm.pdf(x_range, mu_mle_solution, sigma_mle_solution)
plt.plot(x_range, mle_pdf, 'r--', linewidth=2, 
         label=f'MLE: N({mu_mle_solution:.2f}, {sigma_mle_solution:.2f}²)')

plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Parameter Estimation Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Comparison of estimates:")
print(f"μ estimates are identical: {mu_mom_solution:.3f}")
print(f"σ difference: {abs(sigma_mom_solution - sigma_mle_solution):.4f}")
print(f"MoM uses n-1 in denominator (unbiased), MLE uses n (biased but consistent)")

In [ ]:
# SOLUTION - Task 5: Goodness of fit
print("\n✅ SOLUTION - Task 5:")

# Kolmogorov-Smirnov test
ks_stat_sol, ks_p_sol = kstest(mystery_data, 
                               lambda x: stats.norm.cdf(x, mu_mle_solution, sigma_mle_solution))

# Anderson-Darling test
ad_stat_sol, ad_crit_sol, ad_sig_sol = anderson(mystery_data, dist='norm')

print(f"Kolmogorov-Smirnov Test:")
print(f"Test statistic: {ks_stat_sol:.4f}")
print(f"P-value: {ks_p_sol:.4f}")
print(f"Conclusion: {'Reject H0' if ks_p_sol < 0.05 else 'Fail to reject H0'} (normal distribution)")

print(f"\nAnderson-Darling Test:")
print(f"Test statistic: {ad_stat_sol:.4f}")
print(f"Critical value (5%): {ad_crit_sol[2]:.4f}")
print(f"Conclusion: {'Reject' if ad_stat_sol > ad_crit_sol[2] else 'Accept'} normality hypothesis")

print(f"\n🎯 OVERALL CONCLUSION:")
print(f"The mystery data appears to be normally distributed with:")
print(f"• Mean ≈ {mu_mle_solution:.2f}")
print(f"• Standard deviation ≈ {sigma_mle_solution:.2f}")
print(f"• Both MoM and MLE give very similar results")
print(f"• Goodness-of-fit tests support the normal distribution hypothesis")

</details>

## Summary & Key Takeaways

### 🎯 What We Learned Today

1. **Parameter Estimation Fundamentals**
   - The goal: estimate unknown parameters from observed data
   - Two main approaches: Method of Moments (MoM) and Maximum Likelihood Estimation (MLE)

2. **Method of Moments (MoM)**
   - ✅ Simple and intuitive
   - ✅ Always exists (if moments exist)
   - ❌ Not always optimal (higher variance)
   - Best for: quick estimates, starting values, simple distributions

3. **Maximum Likelihood Estimation (MLE)**
   - ✅ Optimal properties (minimum variance, consistency, efficiency)
   - ✅ Strong theoretical foundation
   - ❌ May require numerical optimization
   - Best for: precise estimates, inference, model comparison

4. **Practical Considerations**
   - Always validate your assumptions
   - Use diagnostic tools (Q-Q plots, goodness-of-fit tests)
   - Consider bootstrap for uncertainty quantification
   - Start simple (MoM) then refine (MLE) if needed

### 🚀 Next Steps

- **Advanced Topics**: Bayesian estimation, robust methods, regularization
- **Applications**: Dive deeper into ML models (neural networks, time series)
- **Practice**: Apply these methods to your own datasets

### 📚 Additional Resources

- **Books**: Casella & Berger "Statistical Inference", Lehmann & Casella "Theory of Point Estimation"
- **Software**: Practice with `scipy.optimize`, `statsmodels`, `PyMC` for Bayesian methods
- **Datasets**: UCI ML Repository, Kaggle for real-world practice

---

**🎉 Congratulations!** You've completed the Parameter Estimation Workshop. You now have practical skills in:
- Implementing MoM and MLE estimators
- Comparing different estimation methods
- Validating estimation results
- Applying parameter estimation to real problems

**Questions?** Don't hesitate to ask or explore the additional resources!